In [1]:
from queryAnalyzer import getPlayersInQuery, getLeaguesInQuery, getTeams, getSeasons, getStages
import sqlAnswers

In [2]:
import sqlite3
sqlite_file = 'database.sqlite'

query1 = "Thomas Mueller ist der beste Spieler der Welt!"
query1 = "Müller Thomas ist der beste Spieler der Welt, nicht Aaron Doran!"
query2 = "Bundesliga und Jupiler League sind supi!"
query2 = "Die deutsche Liga ist supi!"


In [3]:
#make query insensitive for umlaute
def preProcessQuery(query):
    umlaut_a = 'ä'.encode()
    umlaut_A = 'Ä'.encode()
    umlaut_o = 'ö'.encode()
    umlaut_O = 'Ö'.encode()
    umlaut_u = 'ü'.encode()
    umlaut_U = 'Ü'.encode()
    
    query = query.encode()
    
    query = query.replace(umlaut_a, b'ae')
    query = query.replace(umlaut_A, b'Ae')
    query = query.replace(umlaut_o, b'oe')
    query = query.replace(umlaut_O, b'Oe')
    query = query.replace(umlaut_u, b'ue')
    query = query.replace(umlaut_U, b'Ue')
    
    query = query.decode('utf-8')
    
    if(query[-1:] == '?' or query[-1:] == '!' or query[-1:] == '.' ):
        return query[:-1]
    return query

Query wird verarbeitet:
    
- Liefert eine playerID eines Spielers zurück
- Keyword "wiegt" (oder auch "schwer") wird gefunden

-> dieses keyword hat nen pointer auf eine SQL Query mit Platzhaltern, in welches die ID des Spielers von dem gesetzt wird


'SELECT weight FROM player WHERE id = playerID'


flair tool 
was passiert wenn der Input da rein kommt

Queries mit "Tags"

Spieler
Ligen
Verein (evtl abkürzung)
Saison / Jahr





In [4]:
def getTeamMatches(): #testausrufe
    import sqlite3
    conn = sqlite3.connect('database.sqlite')
    c = conn.cursor()
    queryText = 'SELECT * FROM match WHERE id = 307'
    c.execute(queryText)
    result = c.fetchall()
    conn.close()
    return result
print(getTeamMatches())

[(307, 1, 1, '2009/2010', 1, '2009-07-31 00:00:00', 665318, 9985, 9997, 2, 2, 1, 2, 4, 6, 8, 2, 4, 6, 8, 4, 6, 1, 2, 4, 6, 8, 2, 4, 6, 8, 4, 6, 1, 3, 3, 3, 3, 7, 7, 7, 7, 10, 10, 1, 3, 3, 3, 3, 7, 7, 7, 7, 10, 10, 38797, 39580, 38800, 37861, 47411, 35412, 39631, 39591, 37262, 25957, 38369, 37868, 33595, 38347, 3329, 149150, 38354, 37866, 36836, None, 5016, 45490, None, None, None, None, None, None, None, None, 1.22, 6, 13, 1.2, 5.5, 10.5, 1.2, 5, 10, 1.17, 5.5, 12, None, None, None, 1.2, 5, 11, 1.18, 6.5, 13, 1.2, 5.5, 12, 1.2, 5.5, 11, 1.17, 6, 11)]


In [5]:
from flair.embeddings import WordEmbeddings
from flair.data import Sentence
german_embedding = WordEmbeddings('de-crawl')
 
import numpy
import json

In [6]:
def getVectorOfAnswer(index):
    with open('trainData.json', 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    embeddings = 0
    for sentence in obj[str(index)]:
        
        sentence = Sentence(preProcessQuery(sentence))
        german_embedding.embed(sentence)
        
        embed = 0
        for token in sentence:
            #print(token)
            embed += token.embedding.numpy()
        #evtl embed durch die anzahl an Wörtern darin teilen ? beim Finalen auch?
        embeddings += embed
    return embeddings
#print(getVectorOfAnswer(0))

In [7]:
import numpy as np
def getVectorOfQuery(query):
    sentence = Sentence(query)
    german_embedding.embed(sentence)
    
    embed = 0
    for token in sentence:
        #print(token)
        embed += token.embedding.numpy()
    return embed


def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))



In [8]:
def getAnswer(question):
    query = preProcessQuery(question)
    queryV = getVectorOfQuery(getPlayersInQuery(query)[1]) # erwarter preprocessed Query, TODO 
    print(getPlayersInQuery(query)[1])

    queryRefined = [getPlayersInQuery(query)[0], getLeaguesInQuery(query),getTeams(query),getSeasons(query),getStages(query)]
#print(queryRefined)
    answers = sqlAnswers.getAnswers()


    queryToCompare = [len(queryRefined[0]),len(queryRefined[1]),len(queryRefined[2]),len(queryRefined[3]),len(queryRefined[4])]
    possibleAnswers = [] 
    #contains index with possible ansers which could match the query depending
    #on number of player occurances etc

    for x in range(0,len(answers)): #check for every "answer", if [1,0,0,0] matches [1,0,0,0] etc
        sqlMatches = True;
        for i in range(0, len(queryToCompare)): #interate through the whole [0,1,2,3] array and check 
            if(queryToCompare[i] != answers[x][i]):  # if something doesnt match, throw it out
                sqlMatches = False
        if(sqlMatches):
            possibleAnswers.append(x)
        

    print("In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: ")       
    print(possibleAnswers)
    if(len(possibleAnswers) == 1):
        return answers[possibleAnswers[0]][5](queryRefined)
    if(len(possibleAnswers) == 0):
        return "Es konnte leider keine passende Antwort gefunden werden!"
    angles = []
    for answerIndex in possibleAnswers:
        angles.append(angle_between(getVectorOfAnswer(answerIndex),queryV))

    print(angles)
    print("Beste gefundene Antwort anhand der Cosinus Ähnlichkeit: " + str(angles.index(min(angles))))

    return answers[angles.index(min(angles))][5](queryRefined)

In [9]:
#Stellen Sie Ihre Frage:
eingabe = input("Ihre Eingabe? ")
print(getAnswer(eingabe))

Ihre Eingabe? Was ist der Heimverein von Manuel Neuer?
Was ist der Heimverein von SPIELER
In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: 
[0, 1, 2, 3]
[0.47503154889938415, 0.5552810544626917, 0.4550696135642723, 0.6079406842618443]
Beste gefundene Antwort anhand der Cosinus Ähnlichkeit: 2
Der bevorzugte Fuß ist rechts.


In [ ]:
Was ist der Heimverein von Manuel Neuer?
s
Wo hat Manuel Neuer 2008/2009 gespielt?
Wo hat Manuel Neuer 2014/2015 gespielt?

was ist der bevorzugte Fuß von Toni Kroos?

Was ist das Geburtsdatum von Jerome Boateng?